**Install Required Packages**

In [0]:
%cd ~

!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-tk

!pip3 install -q Cython contextlib2 pillow lxml matplotlib PyDrive

!pip3 install -q pycocotools

%cd ~/models/research
!protoc object_detection/protos/*.proto --python_out=.

!python3 setup.py build
!python3 setup.py install

%cd slim
!pip3 install -e .

%cd ..
!python3 object_detection/builders/model_builder_test.py

**Mount to Google Drive**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
%cd ~/models/research/object_detection
!mkdir training
%cd training
from google.colab import files
uploaded = files.upload()

/root/models/research/object_detection
mkdir: cannot create directory ‘training’: File exists
/root/models/research/object_detection/training


Saving label_map.pbtxt to label_map.pbtxt
Saving model.ckpt.data-00000-of-00001 to model.ckpt.data-00000-of-00001
Saving model.ckpt.index to model.ckpt.index
Saving model.ckpt.meta to model.ckpt.meta
Saving test.record to test.record
Saving train.record to train.record


In [0]:
%cd ~/models/research/object_detection
!cp training/training/model.ckpt-3844.data-00000-of-00001 training/
!cp training/training/model.ckpt-3844.index training/
!cp training/training/model.ckpt-3844.meta training/
!cp training/training/checkpoint training/

/root/models/research/object_detection


**Re-Train Model**

In [0]:
%cd ~/models/research/object_detection

!python3 model_main.py \
    --pipeline_config_path=training/ssd_mobilenet_v2.config \
    --model_dir=training/training \
    --alsologtostderr \

/root/models/research/object_detection
W0729 08:42:39.463066 140271114938240 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0729 08:42:39.489133 140271114938240 deprecation_wrapper.py:119] From /root/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0729 08:42:39.499073 140271114938240 deprecation_wrapper.py:119] From /root/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0729 08:42:39.509389 140271114938240 deprecation_wrapper.py:119] From model_main.py:109: The name tf.app.run is deprec

**Save Model and Convert to .tflite**

In [0]:
%cd ~/models/research/object_detection
!python3 export_tflite_ssd_graph.py \
    --pipeline_config_path=training/ssd_mobilenet_v2.config \
    --trained_checkpoint_prefix=training/model.ckpt-<TOBEFILLEDIN> \
    --output_directory=training/saved/original \
    --add_postprocessing_op=true
!toco \
    --graph_def_file=training/saved/original/tflite_graph.pb \
    --output_file=training/saved/converted/converted.tflite \
    --input_shapes=1,600,800,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
    --inference_type=FLOAT \
    --allow_custom_ops

/content/gdrive/My Drive/transfer-learning
label_map.pbtxt			model.ckpt.meta		 train.record
model.ckpt.data-00000-of-00001	ssd_mobilenet_v2.config
model.ckpt.index		test.record
